In [30]:
# https://www.tensorflow.org/tutorials/text/word_embeddings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [4]:
vocab_size = 1000
embedding_size = 5

embedding_layer = layers.Embedding(vocab_size, embedding_size)

In [7]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.0275739 , -0.01354999,  0.01476267,  0.0234868 , -0.03942246],
       [ 0.02769193,  0.00878152,  0.03697341,  0.00128802,  0.0141011 ],
       [ 0.00259596, -0.04183214, -0.02993209,  0.00785676, -0.03427104]],
      dtype=float32)

In [8]:
result.shape  # (samples, sequence_length, embedding_size)

TensorShape([3, 5])

### Sideroad - loading raw text into a word vocabulary

In [41]:
FILE_PATHS = ["b:/!DATASETS/oral2013/transcripts_debug/labels.txt", 
              "b:/!DATASETS/PDTSC/transcripts_debug/labels.txt"]
ENCODING = "windows-1250"

In [42]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

In [43]:
labeled_data_sets = []
for i, file_path in enumerate(FILE_PATHS):
    lines_dataset = tf.data.TextLineDataset(file_path)
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    labeled_data_sets.append(labeled_dataset)

In [44]:
for ex, lab in labeled_data_sets[0].take(5):
    print(ex.numpy().decode(ENCODING))

hmm to nevim vůbec co se jak se mi to podařilo
ale já sem si myslela že zejtra nemám rehabilitaci fakt sem vo tom byla přesvědčená a zistila sem že ji mám
ty ji máš každej den
no teď budu mít až do osumnáctýho ji budu mít každej den takový intenzivní a mám pocit
že mě to asi zabije


In [45]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [46]:
for ex, lab in all_labeled_data.take(5):
    print(ex.numpy().decode(ENCODING))

na této fotografii jsou mí rodiče při svatbě bylo to v roce devatenáct set třicet šest no byli to už ne úplně mladí lidé mamince bylo asi třicet tři let a tatínkovi bylo asi třicet osum
dyž sem jednou uvažovala že bych šla ke zpovědi k salvátorovi tak sem si zkusila stáhnout jejich zpovědní zrcadlo a zjistila sem že na to sem asi moc hloupá
tak na téhle fotce jsem já s manželkou sousedy a jedním kamarádem je to výlet důchodců a někde z restaurace p při občerstvení
asi no
no


## Build a vocabulary

In [47]:
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy().decode(ENCODING))
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

3923

## Encode examples

In [48]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [49]:
example_text = next(iter(all_labeled_data))[0].numpy().decode(ENCODING)
print(example_text)

na této fotografii jsou mí rodiče při svatbě bylo to v roce devatenáct set třicet šest no byli to už ne úplně mladí lidé mamince bylo asi třicet tři let a tatínkovi bylo asi třicet osum


In [50]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[281, 3103, 2681, 2332, 2741, 3539, 3522, 2371, 2390, 3704, 3471, 470, 2031, 1186, 2970, 3403, 1452, 3053, 3704, 433, 816, 1032, 2676, 3493, 240, 2390, 3211, 2970, 2664, 226, 3137, 1118, 2390, 3211, 2970, 350]


## Encode the dataset

In [53]:
def encode(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy().decode(ENCODING))
    return encoded_text, label

def encode_map_fn(text, label):
    # py_func doesn't set the shape of the returned tensors.
    encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))
    encoded_text.set_shape([None])
    label.set_shape([])
    
    return encoded_text, label


In [54]:
all_encoded_data = all_labeled_data.map(encode_map_fn)

In [57]:
for ex, lab in all_encoded_data.take(1):
    print(ex, lab)

tf.Tensor(
[ 281 3103 2681 2332 2741 3539 3522 2371 2390 3704 3471  470 2031 1186
 2970 3403 1452 3053 3704  433  816 1032 2676 3493  240 2390 3211 2970
 2664  226 3137 1118 2390 3211 2970  350], shape=(36,), dtype=int64) tf.Tensor(1, shape=(), dtype=int64)
